<a href="https://colab.research.google.com/github/nguyen-nhat-mai/methane-leak-detection/blob/main/Web_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build web app for classifier model

In [ ]:
!pip install streamlit -q

In [ ]:
!pip install streamlit_folium

Reference:

https://towardsdatascience.com/create-an-image-classification-web-app-using-pytorch-and-streamlit-f043ddf00c24#2b4c
https://www.youtube.com/watch?v=NEhrkeF2o_M

In [97]:
# Write the web app
%%writefile app.py
import streamlit as st
from sklearn.preprocessing import MinMaxScaler
from PIL import Image
import time
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
import base64
from io import BytesIO
import altair as alt
import folium
from streamlit_folium import st_folium, folium_static

#---------------------- TO UPDATE WITH FINAL MODEL-----------------------#
   
def preprocess_image(img):
    img = img.resize((64, 64), Image.ANTIALIAS)  # Resize the image
    data = np.array(img)
    data = np.expand_dims(data, axis=-1)
    data = data.astype('float32') / 255.0
    return np.array([data])

def preprocess_location(image_name,df_normalized):
    matching_row = df_normalized[df_normalized['path'].str.contains(image_name)]
    print(matching_row)
    lat = matching_row['lat'].values[0]
    lon = matching_row['lon'].values[0]
    coord_x = matching_row['coord_x'].values[0]
    coord_y = matching_row['coord_y'].values[0]
    return np.array([[lat, lon, coord_x, coord_y]])

def predict(image, image_name, df_normalized):
    # Load model
    model = tf.keras.models.load_model("model.h5")
    # Load data
    loc_data = preprocess_location(image_name,df_normalized)
    img_data = preprocess_image(image)
    # Do inference
    y_pred = model.predict({"location": loc_data, "img": img_data})
    y_prob = y_pred.flatten()*100

    max_idx = np.argmax(y_prob)
    max_prob = y_prob[max_idx]

    y_class = (y_pred >= 0.5).astype(int).flatten()
    max_class = y_class[max_idx]

    # Return the class and corresponding probability
    return [(max_class, max_prob)]

# ------------------------------WEB APP----------------------------------#

# Define a function to create a download link for a given DataFrame
def download_link(df, filename, text):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode('utf-8')).decode()  # base64 encoding
    href = f"data:text/csv;base64,{b64}"
    return f'<a href="{href}" download="{filename}">{text}</a>'

# Get dictionary of geolocation for id_coord
geo_map = pd.read_csv("metadata.csv")
geo_dict = {row['id_coord']: (row['lat'], row['lon']) for _, row in geo_map.iterrows()}

# Get the normalized location
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(geo_map[['lat', 'lon']])
df_normalized = geo_map.copy()
df_normalized[['lat', 'lon']] = scaled_values

# Start the display on web app
st.set_option('deprecation.showfileUploaderEncoding', False)
st.title("CleanR - Detect Methane Leaks")
st.write("")
st.sidebar.header("Menu")
tabs = ["Prediction on uploaded images", "Leak detection map", "Leak track over time by id_coord", "Prediction detail"]
selected_tab = st.sidebar.radio("", tabs)


if selected_tab == "Prediction on uploaded images":
    file_up = st.file_uploader("Upload your images", type=["jpg", "jpeg", "png","tif", "tiff"], accept_multiple_files=True)
    if file_up is not None:
        st.subheader("Prediction on uploaded images")
        all_predictions = pd.DataFrame() # create an empty DataFrame to store all predictions
        for img_file in file_up:
            image = Image.open(img_file)
            image_8bit = image.convert("L")
            st.image(image_8bit, caption=img_file.name, use_column_width=True)
            st.write("")
            with st.spinner('Predicting...'):
                # Get prediction result
                predictions_df = predict(image_8bit,img_file.name.split(".")[0], df_normalized)
                # Append the current predictions to the DataFrame along with the id_coord, date and geolocation
                id_coord = img_file.name.split(".")[0][-7:]
                date = img_file.name.split("_")[0]
                latitude = geo_dict[id_coord][0]
                longitude = geo_dict[id_coord][1]
                predictions_df = [(id_coord,date,latitude,longitude,)+predictions_df[0]]
                all_predictions = all_predictions.append(predictions_df, ignore_index=True)
                # Print out the prediction labels with probability
                st.write("Prediction:", predictions_df[0][4], "-   Probability (%): ", round(predictions_df[0][5],1))
        if not all_predictions.empty:
            all_predictions.columns = ['Id_coord', "Date","Latitude","Longitude",'Label', 'Probability (%)']
            all_predictions.to_csv("predictions.csv", index=False)

if selected_tab == "Leak detection map":
    st.subheader("Leak detection map")
    all_predictions = pd.read_csv("predictions.csv")
    all_predictions['Date'] = pd.to_datetime(all_predictions['Date'], format='%Y%m%d')
    # Create base map that will be centered around the mean latitude and longitude 
    m = folium.Map(location=[all_predictions.Latitude.mean(), all_predictions.Longitude.mean()], 
                 zoom_start=3, control_scale=True)
    # Loop through each row in the dataframe
    for i,row in all_predictions.iterrows():
        # Setup the content of the popup
        iframe = folium.IFrame('id:' + str(row["Id_coord"]))
        # Initialise the popup using the iframe
        popup = folium.Popup(iframe, min_width=300, max_width=300)
        # Add each row to the map
        folium.Marker(location=[row['Latitude'],row['Longitude']],
                  popup = popup, c=row['Id_coord'],
                  icon=folium.Icon(color='red',icon='info-sign')).add_to(m)
    st_data = st_folium(m, width=700)

if selected_tab == "Leak track over time by id_coord":
    st.subheader("Leak track over time by id_coord")
    all_predictions = pd.read_csv("predictions.csv")
    all_predictions['Date'] = pd.to_datetime(all_predictions['Date'], format='%Y%m%d')
    id_coord_list = list(all_predictions['Id_coord'].unique())
    selected_id_coord = st.sidebar.selectbox("Select an id_coord:", id_coord_list)
    filtered_predictions = all_predictions.loc[all_predictions['Id_coord'] == selected_id_coord]
    chart = alt.Chart(filtered_predictions).mark_line(color="#ff2b2b").encode(x='Date',y='Probability (%)')
    st.altair_chart(chart, use_container_width=True)

if selected_tab == "Prediction detail":
    st.subheader("Prediction Detail")
    all_predictions = pd.read_csv("predictions.csv")
    all_predictions['Date'] = pd.to_datetime(all_predictions['Date'], format='%Y%m%d')
    st.markdown(download_link(all_predictions, "prediction.csv", "Download CSV"), unsafe_allow_html=True)
    st.write("")
    st.write(all_predictions)

Overwriting app.py


In [98]:
# Run app.py and made available on a local URL
!streamlit run app.py & npx localtunnel --port 8501

[############......] - refresh-package-json:localtunnel: timing action:finalize

npx: installed 22 in 2.23s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.86.49.100:8501

your url is: https://yellow-views-prove.loca.lt
2023-05-15 12:10:47.209705: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-15 12:10:49.755 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
2023-05-15 12:10:54.697 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
         date  ...                                               path
379  20230101  ...  images/no_plume/20

In [42]:
import streamlit as st
from sklearn.preprocessing import MinMaxScaler
from PIL import Image
import time
import pandas as pd
import numpy as np
import cv2
import rasterio
import tensorflow as tf
import base64
from io import BytesIO
import altair as alt
import folium
from streamlit_folium import st_folium, folium_static

In [43]:
geo_map = pd.read_csv("metadata.csv")
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(geo_map[['lat', 'lon']])
df_normalized = geo_map.copy()
df_normalized[['lat', 'lon']] = scaled_values
df_normalized

,date,id_coord,plume,set,lat,lon,coord_x,coord_y,path
0,20230223,id_6675,yes,train,0.680823,0.776640,24,47,images/plume/20230223_methane_mixing_ratio_id_...
1,20230103,id_2542,yes,train,0.715369,0.933406,42,37,images/plume/20230103_methane_mixing_ratio_id_...
2,20230301,id_6546,yes,train,0.590616,0.820173,58,15,images/plume/20230301_methane_mixing_ratio_id_...
3,20230225,id_6084,yes,train,0.639702,0.803905,28,62,images/plume/20230225_methane_mixing_ratio_id_...
4,20230105,id_2012,yes,train,0.709010,0.638888,59,44,images/plume/20230105_methane_mixing_ratio_id_...
...,...,...,...,...,...,...,...,...,...
425,20230302,id_6658,no,train,0.675970,0.613322,39,36,images/no_plume/20230302_methane_mixing_ratio_...
426,20230218,id_4690,no,train,0.718922,0.636826,29,28,images/no_plume/20230218_methane_mixing_ratio_...
427,20230213,id_2519,no,train,0.839673,0.496243,23,10,images/no_plume/20230213_methane_mixing_ratio_...
428,20230213,id_5510,no,train,0.691034,0.654647,55,54,images/no_plume/20230213_methane_mixing_ratio_...


In [70]:
def preprocess_location(image_name,df_normalized):
    matching_row = df_normalized[df_normalized['path'].str.contains(image_name)]
    lat = matching_row['lat'].values[0]
    lon = matching_row['lon'].values[0]
    coord_x = matching_row['coord_x'].values[0]
    coord_y = matching_row['coord_y'].values[0]
    return np.array([list(lat, lon, coord_x, coord_y)])

In [72]:
print(preprocess_location("20230103_methane_mixing_ratio_id_0438",df_normalized).shape)

(1, 4)


In [49]:
np.array([1,2,3,4])

array([1, 2, 3, 4])